# Outline

1. Some scripts to generate the test cases
2. Scripts to run the tests with grale
3. Maybe something to show visuals?

# Setup & helpers

In [1]:
%matplotlib inline
import grale.images as images
import grale.lenses as lenses
import grale.cosmology as cosmology
import grale.plotutil as plotutil
import grale.multiplane as multiplane
import grale.renderers as renderers
import grale.feedback as feedback
from grale.constants import *
import numpy as np
import matplotlib.pyplot as plt

cosm = cosmology.Cosmology(0.7, 0.3, 0, 0.7)
D = cosm.getAngularDiameterDistance
cosmology.setDefaultCosmology(cosm)

V = lambda x,y: np.array([x,y], dtype=np.double)

plotutil.setDefaultAngularUnit(ANGLE_ARCSEC)
renderers.setDefaultLensPlaneRenderer("threads")
feedback.setDefaultFeedback("notebook")

In [2]:
z_d = [0.5, 1.0, 1.5, 2.0, 2.5, 2.6, 2.7, 2.8, 2.9]
z_s = [3]

In [3]:
def createPlummerGrid(zd, N, areaWidth, plummerWidth, plummerMass):
    xpos = np.linspace(-areaWidth/2, areaWidth/2, N)
    ypos = np.linspace(-areaWidth/2, areaWidth/2, N)

    compLensParams = [ ]
    Dd = D(zd)
    for y in ypos:
        for x in xpos:
            '''
            plummer = lenses.PlummerLens(Dd, { "width": plummerWidth, "mass": plummerMass})
            compLensParams.append({
                "lens": plummer,
                "factor": 1,
                "angle": 0,
                "x": x,
                "y": y
            })
            '''
            compLensParams.append({
                "mass": plummerMass,
                "width": plummerWidth,
                "x": x,
                "y": y
            })
            
    return lenses.MultiplePlummerLens(Dd, compLensParams)

def setPlummerWeights(compLens, factors):
    Dd = compLens.getLensDistance()
    params = compLens.getLensParameters()
    
    newParams = []
    for p, f in zip(params, factors):
        p["factor"] = f
        newParams.append(p)
        
    return lenses.CompositeLens(Dd, newParams)

In [4]:
'''
def getThetas(lens):
    srcRadius = 1*ANGLE_ARCSEC
    lens.setSourceRedshift(z_s[0])
    thetas = []
    shapes = []
                
    imgPlane = lensInfo.getImagePlane()
    for i in range(-2, 3):
        for j in range(-2, 3):
            pos = (i * ANGLE_ARCSEC, j * ANGLE_ARCSEC)
            sourceShape = images.CircularSource(pos, srcRadius, fade=True)
            shapes.append(sourceShape)

    plane = imgPlane.renderImages(shapes)
    imgPoints = imgPlane.segment(plane)
    imgPoints = np.array([p for part in imgPoints for p in part])
    thetas.extend(imgPoints)
            
    return thetas
'''
def getThetas(n):
    x = np.linspace(-30 * ANGLE_ARCSEC, 30 * ANGLE_ARCSEC, n)
    y = np.linspace(-30 * ANGLE_ARCSEC, 30 * ANGLE_ARCSEC, n)
    xx, yy = np.meshgrid(x, y)
    thetas = np.array([xx.flatten(),yy.flatten()]).T
    return thetas

def getThetasArcSec(n):
    x = np.linspace(-30, 30, n)
    y = np.linspace(-30, 30, n)
    xx, yy = np.meshgrid(x, y)
    thetas = np.array([xx.flatten(),yy.flatten()]).T
    return thetas

# Print setup to file for CUDA

See reference notebook for file structure details

In [35]:
def printCuda(lensdata, thetas, name):
    fp = open(name + ".txt", "w")
    
    fp.write(str(ANGLE_ARCSEC) + "\n")

    # Lenses
    numplanes = len(lensdata)
    fp.write(str(numplanes) + '\n')
    for x in lensdata:
        fp.write(str(x[1]) + " ")
    fp.write('\n')
    
    for x in lensdata:
        params = x[0].getLensParameters()
        # print(params)
        fp.write(str(len(params)) + '\n')
        for l in params:
            fp.write(str(l['x']) + " ")
            fp.write(str(l['y']) + " ")
            fp.write("5 ")
            # p = l['lens'].getLensParameters()
            # print(p)
        fp.write('\n')
    # fp.write('\n')
    
    # Source
    fp.write("1\n")
    fp.write(str(z_s[0]) + " \n")
    
    fp.write(str(len(thetas)) + "\n")
    for t in thetas:
        fp.write(str(t[0]) + " ")
        fp.write(str(t[1]) + "\n")
    # fp.write('\n')
    
    for x in lensdata:
        params = x[0].getLensParameters()
        fp.write(str(len(params)) + '\n')
        for l in params:
            fp.write("1.0 ")
        fp.write('\n')
    
    fp.close()

# Create test files CUDA

The tests used for CUDA. More intensive for scaling test purposes.

In [41]:
numthetas = [100, 250, 500, 750, 1000, 2500]
numlenses = [1, 10, 25,50, 75, 100, 250, 500]
numplanes = [1, 2, 4, 8]

# Single lens plane
'''
for t in numthetas:
    thetas = getThetasArcSec(t)
    for s in numlenses:
        lens1 = createPlummerGrid(z_d[0], s, 15*ANGLE_ARCSEC, 5*ANGLE_ARCSEC, 1e8*MASS_SUN)
        lensInfo = plotutil.LensInfo(lens=lens1, zs=z_s[0], zd=z_d[0], size=60*ANGLE_ARCSEC)
        print("S: %d; T: %d" % (s, len(thetas)))
        printCuda([[lens1, z_d[0]]], thetas, "../data/single_%d_%d" % (t, s))
'''
        
# Multiplane
'''
for t in numthetas[:-1]:
    thetas = getThetasArcSec(t)
    for s in numlenses:
        for plane in numplanes:
            lenslist = []
            for x in range(plane):
                lens = createPlummerGrid(z_d[x], s, 15*ANGLE_ARCSEC, 5*ANGLE_ARCSEC, 1e8*MASS_SUN)
                lenslist.append([lens, z_d[x]])
            print("S: %d; x: %d" % (s, plane))
            printCuda(lenslist, thetas, "../data/multi_%ds_%d_%d" % (plane, t, s))
'''

'\nfor t in numthetas[:-1]:\n    thetas = getThetasArcSec(t)\n    for s in numlenses:\n        for plane in numplanes:\n            lenslist = []\n            for x in range(plane):\n                lens = createPlummerGrid(z_d[x], s, 15*ANGLE_ARCSEC, 5*ANGLE_ARCSEC, 1e8*MASS_SUN)\n                lenslist.append([lens, z_d[x]])\n            print("S: %d; x: %d" % (s, plane))\n            printCuda(lenslist, thetas, "../data/multi_%ds_%d_%d" % (plane, t, s))\n'

# Single lens plane tests

Tests using a single lens and source plane.

- 1 lens
- 10 * 10 lenses
- 25 * 25 lenses
- 50 * 50 lenses
- 100 * 100 lenses

- List of 1 000 000 thetas (grid)

In [5]:
Ds = cosm.getAngularDiameterDistance(z_s[0])
Dds = cosm.getAngularDiameterDistance(z_d[0], z_s[0])
sublenscounts = [1, 10, 25, 50, 75, 100, 150, 200]
thetas = getThetas(500)

for s in sublenscounts:
    lens1 = createPlummerGrid(z_d[0], s, 15*ANGLE_ARCSEC, 5*ANGLE_ARCSEC, 1e8*MASS_SUN)
    lensInfo = plotutil.LensInfo(lens=lens1, zs=z_s[0], zd=z_d[0], size=60*ANGLE_ARCSEC)
    print("S: %d; T: %d" % (s, len(thetas)))
    
    %timeit sourceBetas = lens1.traceTheta(Ds, Dds, thetas)
    
    continue
    if s <= 50:
        continue
    break

S: 1; T: 250000
4.39 ms ± 12.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
S: 10; T: 250000
110 ms ± 5.98 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
S: 25; T: 250000
679 ms ± 67.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
S: 50; T: 250000
2.73 s ± 414 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
S: 75; T: 250000
6.13 s ± 813 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
S: 100; T: 250000
10.9 s ± 679 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
S: 150; T: 250000
24.6 s ± 3.19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
S: 200; T: 250000
43.7 s ± 31.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Multiplane tests

No multicore speedup unfortunately.

How to properly run traceTheta, I don't now. Maybe just copying the function from grale slightly modified?

In [6]:
def traceTheta(lz, zs, thetas):
    """For each theta position in `thetas`, calculate the corresponding position
    in the source plane. Note that this is all calculated using a single processor
    core, no speedup using e.g. OpenMP will be performed."""
    import copy

    cosmology = cosm
    sourceRedshift = zs

    alphas = { }
    T = []

    def getAlphas(j):
        if j in alphas:
            return alphas[j]
        a = lz[j][0].getAlphaVector(T[j])
        alphas[j] = a
        return a

    N = len(lz)
    for i in range(0, N):

        Ti = copy.deepcopy(thetas)
        Di = cosmology.getAngularDiameterDistance(lz[i][1])
            
        for j in range(0, i):
            Dji = cosmology.getAngularDiameterDistance(lz[j][1], lz[i][1])
            Ti -= Dji/Di * getAlphas(j)
        
        T.append(Ti)

    # Make sure these are cached
    getAlphas(N-1)

    Ti = copy.deepcopy(thetas)
    Di = cosmology.getAngularDiameterDistance(sourceRedshift)
    for j in range(0, N):
        Dji = cosmology.getAngularDiameterDistance(lz[j][1], sourceRedshift)
        Ti -= Dji/Di * alphas[j]

    return Ti

In [8]:
lensdim = 50
thetas = getThetas(500)

planes = [1, 2, 3, 4, 8]

for plane in planes:
    lenslist = []
    for x in range(plane):
        lens = createPlummerGrid(z_d[x], lensdim, 15*ANGLE_ARCSEC, 5*ANGLE_ARCSEC, 1e8*MASS_SUN)
        lenslist.append([lens, z_d[x]])
    
    print("S: %d; x: %d" % (s, plane))
    %timeit traceTheta(lenslist, z_s[0], thetas)


S: 200; x: 1
2.73 s ± 939 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
S: 200; x: 2
5.46 s ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
S: 200; x: 3
8.19 s ± 2.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
S: 200; x: 4
10.9 s ± 3.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
S: 200; x: 8
21.9 s ± 5.91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
